In [ ]:
import pandas as pd
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chisquare, chi2

factors={'發生年度': 0,
 '發生時間': 1,
 '天候名稱': 2,
 '光線名稱': 3,
 '事故類型及型態大類別名稱': 4,
 '事故類型及型態子類別名稱': 5,
 '肇因研判大類別名稱-主要': 6,
 '肇因研判子類別名稱-主要': 7,
 '死亡受傷人數': 8,
 '當事者區分-類別-大類別名稱-車種': 9,
 '當事者屬-性-別名稱': 10,
 '當事者事故發生時年齡': 11,
 '當事者行動狀態大類別名稱': 12,
 '當事者行動狀態子類別名稱': 13,
 '車輛撞擊部位大類別名稱-最初': 14,
 '車輛撞擊部位子類別名稱-最初': 15,
 '肇因研判大類別名稱-個別': 16,
 '肇因研判子類別名稱-個別': 17}

In [ ]:
#read all files in the folder A2
#save the data in a dataframe
file_list=os.listdir('A2')
df=pd.DataFrame()
for file in file_list:
    data=pd.read_csv('A2/'+file,header=None)
    df=pd.concat([df,data],axis=0)
#do the same for the files in folder A1
file_list=os.listdir('A1')
df1=pd.DataFrame()
for file in file_list:
    data=pd.read_csv('A1/'+file,header=None)
    df1=pd.concat([df1,data],axis=0)
d_A1={}
for i in range(len(df1)):
    key=str(df1.iloc[i,2])+'_'+str(df1.iloc[i,3])+'_'+str(df1.iloc[i,6])
    value=[df1.iloc[i,0],df1.iloc[i,3],df1.iloc[i,7],df1.iloc[i,8],df1.iloc[i,28],df1.iloc[i,29],df1.iloc[i,30],df1.iloc[i,31],df1.iloc[i,32],df1.iloc[i,34],df1.iloc[i,36],df1.iloc[i,37],df1.iloc[i,40],df1.iloc[i,41],df1.iloc[i,42],df1.iloc[i,43],df1.iloc[i,46],df1.iloc[i,47]]
    if key not in d_A1 and key!='發生日期_發生時間_發生地點' and key!= '發生年度_發生時間_發生地點':
        d_A1[key]=value
d_A2={}
#key is the 3,4,7 column, value is the 3,4,8,9,29,30,31,32,33,35,37,38,41,42,43,44,47,48 column
for i in range(len(df)):
    # key=str(df.iloc[i,3])+'_'+str(df.iloc[i,4])+'_'+str(df.iloc[i,7])
    # value=[df.iloc[i,1],df.iloc[i,4],df.iloc[i,8],df.iloc[i,9],df.iloc[i,29],df.iloc[i,30],df.iloc[i,31],df.iloc[i,32],df.iloc[i,33],df.iloc[i,35],df.iloc[i,37],df.iloc[i,38],df.iloc[i,41],df.iloc[i,42],df.iloc[i,43],df.iloc[i,44],df.iloc[i,47],df.iloc[i,48]]
    key=str(df.iloc[i,2])+'_'+str(df.iloc[i,3])+'_'+str(df.iloc[i,6])
    value=[df.iloc[i,0],df.iloc[i,3],df.iloc[i,7],df.iloc[i,8],df.iloc[i,28],df.iloc[i,29],df.iloc[i,30],df.iloc[i,31],df.iloc[i,32],df.iloc[i,34],df.iloc[i,36],df.iloc[i,37],df.iloc[i,40],df.iloc[i,41],df.iloc[i,42],df.iloc[i,43],df.iloc[i,46],df.iloc[i,47]]
    if key not in d_A2 and key!='發生日期_發生時間_發生地點' and key!= '發生年度_發生時間_發生地點':
        d_A2[key]=value
#save the dictionary to a json file
with open('A1.json','w',encoding='utf-8') as f:
    json.dump(d_A1,f)
with open('A2.json','w',encoding='utf-8') as f:
    json.dump(d_A2,f)

In [ ]:
#read the json file
with open('A1.json','r',encoding='utf-8') as f:
    d_A1=json.load(f)
with open('A2.json','r',encoding='utf-8') as f:
    d_A2=json.load(f)
try:
    del d_A1['nan_nan_nan']
    del d_A2['nan_nan_nan']
    # print('delete nan_nan_nan')
except:
    pass
with open('d_A1_p2.json','r',encoding='utf-8') as f:
    d_A1_p2=json.load(f)
with open('d_A2_p2.json','r',encoding='utf-8') as f:
    d_A2_p2=json.load(f)

In [ ]:
len(d_A1),len(d_A2)

In [ ]:
len(d_A1_p2),len(d_A2_p2)

In [ ]:
def casulty(str0):
    #process the string '死亡1;受傷1' to get the number of casulty as a list [1,1]
    casulty_list=[0,0]
    str0=str(str0)
    if str0=='':
        return casulty_list
    elif str0=='死亡受傷人數':
        return ['死亡人數','受傷人數']
    else:
        casulty=str0.split(';')
        for c in casulty:
            if c[0]=='死':
                casulty_list[0]=c[2:]
            if c[0]=='受':
                casulty_list[1]=c[2:]
        return casulty_list
def time_h(str0):
    t=int(str0)
    return t//10000
    # if str0=='':
        # return -1
    # if str0=='發生時間':
        # return '發生時間'
    # if not str(str0).isdigit():
        # return -1
    # if str(str0)[:-4]=='':
        # return -1
    # return str(str0)[:-4]
def time_analysis(d):
    times={}
    for i in d:
        time=int(time_h(d[i][1]))
        times[time]=times.get(time,0)+1
    return times

def casulty_analysis(d):
    death=0
    injury=0
    for i in d:
        casultys=casulty(d[i][8])
        if casultys[0]=='':
            print(i)
            continue
        death+=int(casultys[0])
        injury+=int(casultys[1])
    return death,injury
def year_analysis(d):
    years={}
    for i in d:
        year=str(d[i][0])
        if not year.isdigit():
            continue    
        years[year]=years.get(year,0)+1
    return years
def casulty_year_analysis(d):
    death_year={}
    injury_year={}
    for i in d:
        year=str(d[i][0])
        if not year.isdigit():
            continue
        casultys=casulty(d[i][8])
        death_year[year]=death_year.get(year,0)+int(casultys[0])
        injury_year[year]=injury_year.get(year,0)+int(casultys[1])
    return death_year,injury_year
def one_factor(factor,d):
    factor_dict={}
    for i in d:
        factor_name=d[i][factors[factor]]
        factor_dict[factor_name]=factor_dict.get(factor_name,0)+1
    return factor_dict
def two_factors(factor1,factor2,d):
    factor_dict={}
    for i in d:
        factor_name1=d[i][factors[factor1]]
        factor_name2=d[i][factors[factor2]]
        factor_dict[factor_name1]=factor_dict.get(factor_name1,{})
        factor_dict[factor_name1][factor_name2]=factor_dict[factor_name1].get(factor_name2,0)+1
    return factor_dict
def one_factor_person(factor1,factor2,d):
    factor_dict={}
    for i in d:
        factor_name1=str(d[i][factors[factor1]])
        factor_name2=str(d[i][factors[factor2]])
        factor_dict[factor_name1+'_'+factor_name2]=factor_dict.get(factor_name1+'_'+factor_name2,0)+1
    return factor_dict

def two_factors_person(factor1,factor2a,factor2b,d):
    factor_dict={}
    for i in d:
        factor1_name=d[i][factors[factor1]]
        factor2_name1=str(d[i][factors[factor2a]])
        factor2_name2=str(d[i][factors[factor2b]])
        factor_dict[factor1_name]=factor_dict.get(factor1_name,{})
        factor_dict[factor1_name][factor2_name1+'_'+factor2_name2]=factor_dict[factor1_name].get(factor2_name1+'_'+factor2_name2,0)+1
    return factor_dict

def factor_death(factor,d):
    factor_dict={}
    for i in d:
        factor_name=d[i][factors[factor]]
        casultys=casulty(d[i][8])
        factor_dict[factor_name]=factor_dict.get(factor_name,0)+int(casultys[0])
    return factor_dict

def factor_injury(factor,d):
    factor_dict={}
    for i in d:
        factor_name=d[i][factors[factor]]
        casultys=casulty(d[i][8])
        factor_dict[factor_name]=factor_dict.get(factor_name,0)+int(casultys[1])
    return factor_dict

#split the age into groups: 17-, 18-24, 25-34, 35-44, 45-54, 55-64, 65+
def age_group(age):
    if age=='' or age==0:
        return '未知'
    if not str(age).isdigit():
        return '未知'
    age=int(age)
    if age<18:
        return '17-'
    if age<25:
        return '18-24'
    if age<35:
        return '25-34'
    if age<45:
        return '35-44'
    if age<55:
        return '45-54'
    if age<65:
        return '55-64'
    return '65+'

def age_group_analysis(d):
    age_group_dict={}
    for i in d:
        age=age_group(d[i][11])
        age_group_dict[age]=age_group_dict.get(age,0)+1
    return age_group_dict

def consolidate_vehicle_types(data):
    """
    將車種資料進行整併。

    :param data: dict, 包含車種類別及其對應數量的字典
    :return: dict, 整併後的車種類別及其對應數量
    """

    # 初始化整併後的車種類型字典
    consolidated_data = {
        '機慢車': 0,
        '小客車': 0,
        '貨車': 0,
        '大客車': 0,
        '其他車': 0,
        '人': 0,
        '未知或未分類': 0
    }

    # 定義車種映射
    vehicle_type_mapping = {
        '機車': '機慢車',
        '慢車': '機慢車',

        '小客車': '小客車',
        '小客車(含客、貨兩用)': '小客車',

        '小貨車(含客、貨兩用)': '貨車',
        '小貨車': '貨車',
        '大貨車': '貨車',
        '曳引車': '貨車',
        '半聯結車': '貨車',
        '全聯結車': '貨車',

        '大客車': '大客車',

        '其他車': '其他車',
        '特種車': '其他車',
        '軍車': '其他車',

        '人': '人'
    }

    # 整併數據
    for vehicle_type, count in data.items():
        # 如果車種為空值，則加入到未知或未分類類別
        if pd.isna(vehicle_type):
            consolidated_data['未知或未分類'] += count
            continue

        # 確認車種映射並更新相應類型的數量
        if vehicle_type in vehicle_type_mapping:
            consolidated_data[vehicle_type_mapping[vehicle_type]] += count
        else:
            consolidated_data['未知或未分類'] += count

    return consolidated_data

def simplify_accident_types(data):
    """
    將事故類型進行分類和簡化。
    
    :param data: dict, 包含事故類型及其對應數量的字典
    :return: dict, 簡化後的事故類型及其對應數量
    """
    
    # 初始化簡化類型的字典
    simplified_data = {
        '撞擊事故': 0,
        '車與車事故_側撞': 0,
        '車與車事故_擦撞': 0,
        '車與車事故_對撞': 0,
        '車與車事故_追撞': 0,
        '車與車事故_其他': 0,
        '汽(機)車本身事故_路上翻車、摔倒': 0,
        '汽(機)車本身事故_衝出路外': 0,
        '汽(機)車本身事故_其他': 0,
        '行人事故': 0,
        '平交道事故': 0,
        '其他': 0
    }
    
    # 定義類型映射
    type_mapping = {
        '汽(機)車本身_撞號誌、標誌桿': '撞擊事故',
        '汽(機)車本身_撞路樹、電桿': '撞擊事故',
        '汽(機)車本身_撞橋樑、建築物': '撞擊事故',
        '汽(機)車本身_撞交通島': '撞擊事故',
        '汽(機)車本身_撞護欄(樁)': '撞擊事故',
        '車輛本身_撞電桿': '撞擊事故',
        '車輛本身_撞路樹': '撞擊事故',
        '車輛本身_撞號誌、標誌桿': '撞擊事故',
        '車輛本身_撞護欄(樁)': '撞擊事故',
        '車輛本身_撞建築物': '撞擊事故',
        '車輛本身_撞橋樑(橋墩)': '撞擊事故',
        '車輛本身_撞工程施工': '撞擊事故',
        '汽(機)車本身_撞非固定設施': '撞擊事故',
        '汽(機)車本身_撞動物': '撞擊事故',
        '車輛本身_撞動物': '撞擊事故',
        '車輛本身_撞非固定設施': '撞擊事故',

        '車與車_側撞': '車與車事故_側撞',
        '車與車_同向擦撞': '車與車事故_擦撞',
        '車與車_對向擦撞': '車與車事故_擦撞',
        '車與車_對撞': '車與車事故_對撞',
        '車與車_追撞': '車與車事故_追撞',
        '車與車_倒車撞': '車與車事故_其他',
        '車與車_路口交岔撞': '車與車事故_其他',
        '車與車_其他': '車與車事故_其他',
        
        '汽(機)車本身_衝出路外': '汽(機)車本身事故_衝出路外',
        '汽(機)車本身_路上翻車、摔倒': '汽(機)車本身事故_路上翻車、摔倒',
        '車輛本身_衝出路外': '汽(機)車本身事故_衝出路外',
        '車輛本身_路上翻車、摔倒': '汽(機)車本身事故_路上翻車、摔倒',
        '汽(機)車本身_其他': '汽(機)車本身事故_其他',
        '車輛本身_其他': '汽(機)車本身事故_其他',
        
        '人與汽(機)車_其他': '行人事故',
        '人與汽(機)車_同向通行中': '行人事故',
        '人與汽(機)車_穿越道路中': '行人事故',
        '人與汽(機)車_對向通行中': '行人事故',
        '人與汽(機)車_佇立路邊(外)': '行人事故',
        '人與汽(機)車_衝進路中': '行人事故',
        '人與汽(機)車_從停車後(或中)穿出': '行人事故',
        '人與汽(機)車_在路上作業中': '行人事故',
        '人與汽(機)車_在路上嬉戲': '行人事故',
        '人與車_穿越道路中': '行人事故',
        '人與車_同向通行中': '行人事故',
        '人與車_其他': '行人事故',
        '人與車_從停車後(或中)穿出': '行人事故',
        '人與車_對向通行中': '行人事故',
        '人與車_衝進路中': '行人事故',
        '人與車_在路上作業中': '行人事故',
        '人與車_佇立路邊(外)': '行人事故',
        
        '平交道事故_正越過平交道中': '平交道事故',
        '平交道事故_衝過(或撞壞)遮斷器': '平交道事故',
        '平交道事故_其他': '平交道事故',
        '平交道事故_暫停位置不當': '平交道事故',
    }
    
    # 將數據按類型歸類
    for accident_type, count in data.items():
        # 如果類型為空值，則跳過
        if pd.isna(accident_type):
            continue
        
        # 確認類型映射並更新相應類型的數量
        if accident_type in type_mapping:
            simplified_data[type_mapping[accident_type]] += count
        else:
            simplified_data['其他'] += count

    return simplified_data

def consolidate_vehicle_status(data):
    """
    將事故資料中的車輛及行人的狀態進行整併。

    :param data: dict, 包含狀態類別及其對應數量的字典
    :return: dict, 整併後的狀態類別及其對應數量
    """
    
    # 初始化整併後的類型字典
    consolidated_data = {
        '直行': 0,
        '轉彎': 0,
        '變換車道': 0,
        '車輛靜止或停車': 0,
        '行駛不當': 0,
        '急減速或急停止': 0,
        '人行動作': 0,
        '其他': 0
    }
    
    # 定義狀態映射
    status_mapping = {
        '車的狀態_向前直行中': '直行',
        
        '車的狀態_左轉彎': '轉彎',
        '車的狀態_右轉彎': '轉彎',
        
        '車的狀態_向右變換車道': '變換車道',
        '車的狀態_向左變換車道': '變換車道',
        
        '車的狀態_靜止(引擎熄火)': '車輛靜止或停車',
        '車的狀態_等待(引擎未熄火)': '車輛靜止或停車',
        '車的狀態_停車操作中': '車輛靜止或停車',

        '車的狀態_起步': '行駛不當',
        '車的狀態_插入行列': '行駛不當',
        '車的狀態_倒車': '行駛不當',
        '車的狀態_超車(含超越)': '行駛不當',
        '車的狀態_迴轉或橫越道路中': '行駛不當',
        
        '車的狀態_急減速或急停止': '急減速或急停止',
        
        '人的狀態_步行': '人行動作',
        '人的狀態_靜立(止)': '人行動作',
        '人的狀態_奔跑': '人行動作',

        '人的狀態_其他': '其他',
        '車的狀態_其他': '其他',
        '人的狀態_不明': '其他',
        'nan_nan': '其他',
        '不明_不明': '其他'
    }
    
    # 整併數據
    for status, count in data.items():
        # 如果狀態為空值，則跳過
        if pd.isna(status):
            continue
        
        # 確認狀態映射並更新相應類型的數量
        if status in status_mapping:
            consolidated_data[status_mapping[status]] += count
        else:
            consolidated_data['其他'] += count

    return consolidated_data

def consolidate_accidents_cause(data):
    # Define the mapping of detailed causes to categories
    category_mapping = {'未注意安全': ['其他_未注意車前狀態',
        '無(非車輛駕駛人因素)_未注意車前狀態',
        '駕駛者_乘客、車上動(生)物干擾分心駕駛',
        '駕駛人_橫越道路不慎',
        '駕駛者_橫越道路不慎',
        '駕駛人_未注意車前狀態',
        '駕駛者_恍神、緊張、心不在焉分心駕駛',
        '駕駛者_觀看其他事故、活動、道路環境或車外資訊分心駕駛',
        '駕駛者_操作、觀看行車輔助或娛樂性顯示設備',
        '駕駛人_起步未注意其他車(人)安全',
        '駕駛者_起步時未注意安全',
        '駕駛者_停車操作時未注意安全',
        '駕駛人_停車操作時，未注意其他車(人)安全',
        '駕駛者_未保持行車安全距離',
        '其他_未保持行車安全距離',
        '駕駛人_未保持行車安全間隔',
        '駕駛者_未保持行車安全間隔',
        '駕駛人_未保持行車安全距離',
        '駕駛者_未靠右行駛',
        '駕駛人_未靠右行駛',
        '駕駛人_倒車未依規定',
        '駕駛者_倒車未依規定'],
        '不當變換車道或方向': ['駕駛人_變換車道或方向不當',
        '駕駛者_變換車道不當',
        '駕駛者_違反禁止變換車道標線',
        '駕駛者_閃避不當(慎)',
        '駕駛者_方向不定(不包括危險駕車)',
        '駕駛人_蛇行、方向不定',
        '其他_方向不定(不包括危險駕車)'],
        '違規行為': ['駕駛人_未依規定讓車',
        '駕駛者_其他未依規定讓車',
        '駕駛者_違反行人專用標誌(線)',
        '駕駛者_未遵守依法令授權交通指揮人員之指揮',
        '駕駛人_違反號誌管制或指揮',
        '駕駛者_違反其他標誌(線)禁制',
        '駕駛者_車輛未依規定暫停讓行人先行',
        '駕駛者_違反禁止左轉、右轉標誌',
        '駕駛者_違反閃光號誌',
        '駕駛者_違反車輛改道標誌',
        '駕駛者_違反禁止迴轉或迴車標誌',
        '駕駛者_違反禁止超車標誌(線)',
        '駕駛者_違反禁止進入標誌',
        '駕駛者_違反車輛專用標誌(線)',
        '駕駛者_違反遵行方向標誌(線)',
        '駕駛者_違反禁行車種標誌(字)',
        '駕駛者_違反禁止各種車輛進入標誌',
        '駕駛人_違反特定標誌(線)禁制',
        '駕駛者_違反其他號誌',
        '駕駛者_違反二段式左(右)轉標誌(線)',
        '駕駛者_行經圓環未依規定讓車',
        '駕駛者_山路會車，靠山壁車未讓外緣車先行',
        '駕駛者_峻狹坡路會車，下坡車未讓上坡車先行',
        '駕駛者_多車道迴轉，未先駛入內側車道',
        '駕駛者_未避讓(跟隨、併駛、超車)消防、救護、警備、工程救險車、毒性化學物質災害事故應變車等執行緊急任務車',
        '駕駛者_違反禁止會車標誌',
        '駕駛人_違規超車',
        '駕駛者_違規超車',
        '駕駛者_闖紅燈直行',
        '駕駛者_闖紅燈左轉（或迴轉）',
        '駕駛者_闖紅燈左轉(或迴轉)',
        '駕駛者_闖紅燈右轉',
        '駕駛者_無號誌路口，少線道未讓多線道先行',
        '駕駛者_無號誌路口，支線道未讓幹線道先行',
        '駕駛者_無號誌路口，左方車未讓右方車先行',
        '駕駛者_無號誌路口，轉彎車未讓直行車先行',
        '駕駛者_有號誌路口，轉彎車未讓直行車先行',
        '無(車輛駕駛者因素)_有號誌路口，轉彎車未讓直行車先行',
        '駕駛人_左轉彎未依規定',
        '駕駛者_左轉彎未依規定',
        '駕駛者_右轉彎未依規定',
        '駕駛人_右轉彎未依規定',
        '駕駛人_迴轉未依規定',
        '駕駛者_迴轉未依規定',
        '燈光_未依規定使用燈光',
        '其他_未保持行車安全間隔',
        '駕駛者_未避讓(跟隨、併駛、超車)消防、救護、警備、工程救險車、毒性化學物質災害事故應變車等執行緊急任務車',
        '駕駛者_未遵守依法令授權交通指揮人員之指揮',
        '駕駛者_使用車輛自動駕駛或先進駕駛輔助系統設備(裝置)不符規定',
        '駕駛人_違規停車或暫停不當而肇事',
        '其他_違規停車或暫停不當而肇事',
        '其他_違規(臨時)停車'],
        '搶快、超速或未依規定減速': ['駕駛者_搶(闖)越平交道',
        '駕駛人_搶(闖)越平交道',
        '駕駛人_爭(搶)道行駛',
        '駕駛者_爭(搶)道行駛',
        '駕駛人_搶越行人穿越道',
        '駕駛人_超速失控',
        '駕駛者_超速駕駛',
        '駕駛人_未依規定減速',
        '駕駛者_未依規定減速'],
        '逆向行駛': ['駕駛人_逆向行駛', '駕駛者_逆向行駛'],
        '危險駕駛': ['駕駛者_其他不當駕車行為',
        '駕駛者_危險駕駛',
        '駕駛人_危險駕駛',
        '駕駛人_酒醉(後)駕駛失控',
        '駕駛者_酒醉(後)駕駛',
        '駕駛者_吸食違禁物駕駛',
        '駕駛人_吸食違禁物後駕駛失控',
        '駕駛人_疲勞(患病)駕駛失控',
        '駕駛者_打瞌睡或疲勞駕駛(包括連續駕車8小時)',
        '駕駛者_患病或服用藥物(疲勞)駕駛',
        '其他_使用手持行動電話失控',
        '駕駛者_飲食、抽(點)菸、拿(撿)物品分心駕駛',
        '駕駛者_使用手持行動電話',
        '駕駛人_其他引起事故之違規或不當行為',
        '非駕駛者_其他不當駕車行為'],
        '行人違規': ['行人(或乘客)_穿越道路未注意左右來車',
        '非駕駛者_穿越道路未注意左右來車',
        '非駕駛者_未依號誌或手勢指揮(示)穿越道路',
        '非駕駛者_未依標誌或標線穿越道路',
        '行人(或乘客)_未依標誌、標線、號誌或手勢指揮穿越道路',
        '行人(或乘客)_其他引起事故之疏失或行為',
        '行人(或乘客)_在道路上嬉戲或奔走不定',
        '行人(或乘客)_未依規定行走行人穿越道、地下道、天橋而穿越道路',
        '非駕駛者_在道路上嬉戲或奔走不定',
        '非駕駛者_在道路上工作之人員未設適當標識',
        '非駕駛者_未依規定行走地下道、天橋穿越道路'],
        '車輛問題': ['機件_車輪脫落或輪胎爆裂',
        '機件_車輛零件脫落',
        '機件_煞車失靈',
        '機件_煞車失靈或故障',
        '機件_方向操縱系統故障',
        '機件_其他引起事故之故障',
        '機件_燈光系統故障',
        '機件_其他機件失靈或故障',
        '機件_車輛附屬機具或車門未盡安全措施'],
        '道路、環境、天候因素': ['無(物或動物)_強風、暴雨、濃霧(煙)',
        '無(非車輛駕駛人因素)_強風、暴雨、濃霧(煙)',
        '交通管制(設施)_路況危險無安全（警告）設施',
        '交通管制(設施)_在路上工作未設適當標識',
        '交通管制(設施)_平交道看守疏失或未放柵欄',
        '交通管制(設施)_施工安全防護措施未依規定或未盡完善(備)',
        '交通管制(設施)_交通管制設施失靈或損毀',
        '交通管制(設施)_路況危險無安全(警告)設施',
        '交通管制(設施)_其他交通管制不當',
        '交通管制(設施)_交通指揮不當',
        '行人(或乘客)_在路上工作未設適當標識',
        '行人(或乘客)_路況危險無安全(警告)設施無(非車輛駕駛人因素)_道路設施(備)、植栽或其他裝置，倒塌或掉(斷)落',
        '無(物或動物)_道路設施(備)、植栽或其他裝置，倒塌或掉(斷)落'],
        '光線不佳': ['其他_因光線、視線遮蔽致生事故', '燈光_夜間行駛無燈光設備', '燈光_暗處停車無燈光、標識'],
        '動物': ['無(非車輛駕駛人因素)_動物竄出', '無(物或動物)_動物竄出', '行人(或乘客)_動物竄出'],
        '其他': ['駕駛者_使用車輛自動駕駛或先進駕駛輔助系統設備(裝置)不符規定',
        '其他_拋錨未採安全措施',
        '其他_車輛拋錨未採安全措施',
        '裝載_其他裝載不當肇事',
        '裝載_其他裝載不當',
        '裝載_載貨超重而失控',
        '裝載_裝載未盡安全措施',
        '裝載_貨物超長、寬、高而肇事',
        '裝載_載運貨物超長、寬、高',
        '裝載_裝卸貨物不當',
        '裝載_未待乘客安全上下開車',
        '裝載_超載人員而失控',
        '裝載_未待乘客安全上下而開車',
        '裝載_超載人員',
        '裝載_裝載貨物不穩妥',
        '裝載_載運貨物超重',
        '非駕駛者_其他引起事故之疏失或行為',
        '其他_發生事故後，未採取安全措施',
        '其他_車輛未停妥滑動致生事故',
        '其他_被車輛輾壓之不明物體彈飛',
        '無(非車輛駕駛人因素)_物品(件)滾(滑行)或飛(掉)落',
        '無(物或動物)_物品(件)滾(滑行)或飛(掉)落',
        '非駕駛者_未待車輛停妥而上下車',
        '其他_開啟車門不當',
        '其他_開啟車門不當而肇事',
        '其他_開啟或關閉車門不當',
        '行人(或乘客)_未待車輛停妥而上下車',
        '非駕駛者_上下車輛時未注意安全',
        '非駕駛者_開啟或關閉車門不當',
        '非駕駛者_乘坐不當(慎)',
        '行人(或乘客)_上下車輛未注意安全',
        '行人(或乘客)_乘坐不當而跌落',
        '駕駛者_其他引起事故之疏失或行為',
        '其他_其他引起事故之違規或不當行為',
        '其他_其他引起事故之疏失或行為',
        '行人(或乘客)_其他引起事故之疏失或行為',
        '其他_車輛或機械操作不當(慎)',
        '其他_其他不當駕車行為'],
        '不明': ['其他_不明原因肇事',
        '駕駛人_不明原因肇事',
        '其他_肇事逃逸未查獲，無法查明肇因',
        '無(非車輛駕駛人因素)_不明原因肇事',
        '行人(或乘客)_尚未發現肇事因素',
        'nan_nan',
        '無(車輛駕駛者因素)_尚未發現肇事因素',
        '非駕駛者_事故發生時當事者逕自離開現場',
        '其他_相關跡證不足且無具體影像紀錄，當事人各執一詞，經分析後無法釐清肇事原因',
        '無(車輛駕駛人因素)_尚未發現肇事因素',
        '無(非車輛駕駛人因素)_尚未發現肇事因素']}
    
    consolidated_data = {}
    
    # Iterate through the original data
    for key, value in data.items():
        if pd.isna(key):
            key = '其他'
        consolidated = '其他'  # Default category
        for category, causes in category_mapping.items():
            if any(cause in key for cause in causes):
                consolidated = category
                break
        if consolidated in consolidated_data:
            consolidated_data[consolidated] += value
        else:
            consolidated_data[consolidated] = value
    
    return consolidated_data

def factor_diff_significance(d1,d2,print_oe=False):
    d1_copy=d1.copy()
    d2_copy=d2.copy()
    rm=[]
    rm5=[]
    for key in d1:
        if key not in d2:
            rm.append(key)
        elif d1[key]*sum(d2.values())/sum(d1.values())<5:
            rm5.append(key)
    for key in rm:
        del d1_copy[key]
    for key in rm5:
        del d1_copy[key]
        del d2_copy[key]
    # sort the dictionary by key to ensure the order of the keys
    d1_copy = dict(sorted(d1_copy.items()))
    d2_copy = dict(sorted(d2_copy.items()))
    # conduct chi-square test
    observed_values = np.array(list(d2_copy.values()))*100/sum(d2_copy.values())
    expected_values = np.array(list(d1_copy.values()))*100/sum(d1_copy.values())
    if print_oe:
        print(f'觀察值：{observed_values}')
        print(f'期望值：{expected_values}')
    chi_square_stat, p_value = chisquare(observed_values, f_exp=expected_values)
    chi_square_stat_threshold = chi2.ppf(0.95, len(observed_values)-1)
    if chi_square_stat==0:
        return f"The observed values is {observed_values} and the expected values is {expected_values}"
    elif p_value<0.05:
        for key in d2_copy:
            if d2_copy[key]>=d1_copy[key]*sum(d2_copy.values())/sum(d1_copy.values())+10:
                print(f'{key}的比例多了{d2_copy[key]-d1_copy[key]*sum(d2_copy.values())/sum(d1_copy.values()):.2f}%')
        return f'有顯著差異，p-value為{p_value:.4f}'# +'，卡方統計量為{chi_square_stat:.4f}<{chi_square_stat_threshold:.4f}'
    else:
        return f'無顯著差異，p-value為{p_value:.4f}'# +'，卡方統計量為{chi_square_stat:.4f}>{chi_square_stat_threshold:.4f}'

def merge_count(*d):
    """
    合併多個字典的計數。
    
    :param *d: dict, 需要合併的字典
    :return: dict, 合併後的計數
    """
    
    # 初始化合併後的字典
    merged_count = {}
    
    # 合併計數
    for data in d:
        for key, value in data.items():
            merged_count[key] = merged_count.get(key, 0) + value
    
    return merged_count

In [ ]:
A1_time=time_analysis(d_A1)
A2_time=time_analysis(d_A2)
print(A1_time,A2_time,sep='\n')

In [ ]:
A1_year=year_analysis(d_A1)
A2_year=year_analysis(d_A2)
print(A1_year)
print(A2_year)

In [ ]:
A1_death,A1_injury=casulty_analysis(d_A1)
A2_death,A2_injury=casulty_analysis(d_A2)
print(A1_death,A1_injury)
print(A2_death,A2_injury)

In [ ]:
A1_death_year,A1_injury_year=casulty_year_analysis(d_A1)
A2_death_year,A2_injury_year=casulty_year_analysis(d_A2)
print(A1_death_year,A1_injury_year,sep='\n')
print(A2_death_year,A2_injury_year,sep='\n')

In [ ]:
A1_vehicle=one_factor('當事者區分-類別-大類別名稱-車種',d_A1)
A2_vehicle=one_factor('當事者區分-類別-大類別名稱-車種',d_A2)
A1_vehicle_simple=consolidate_vehicle_types(A1_vehicle)
A2_vehicle_simple=consolidate_vehicle_types(A2_vehicle)
print(A1_vehicle_simple)
print(A2_vehicle_simple)

In [ ]:
A1_year_vehicle=two_factors('發生年度','當事者區分-類別-大類別名稱-車種',d_A1)
A2_year_vehicle=two_factors('發生年度','當事者區分-類別-大類別名稱-車種',d_A2)
simple_A1_vehicle = {i:consolidate_vehicle_types(A1_year_vehicle[i]) for i in A1_year_vehicle.keys()}
print(simple_A1_vehicle)
simple_A2_vehicle = {i:consolidate_vehicle_types(A2_year_vehicle[i]) for i in A2_year_vehicle.keys()}
print(simple_A2_vehicle)

In [ ]:
A1_vehicle_death=factor_death('當事者區分-類別-大類別名稱-車種',d_A1)
A2_vehicle_death=factor_death('當事者區分-類別-大類別名稱-車種',d_A2)
A1_vehicle_injury=factor_injury('當事者區分-類別-大類別名稱-車種',d_A1)
A2_vehicle_injury=factor_injury('當事者區分-類別-大類別名稱-車種',d_A2)
simple_A1_vehicle_death = consolidate_vehicle_types(A1_vehicle_death)
print(simple_A1_vehicle_death)
simple_A2_vehicle_injury = consolidate_vehicle_types(A2_vehicle_injury)
print(simple_A2_vehicle_injury)

In [ ]:
A1_age_group=age_group_analysis(d_A1)
A2_age_group=age_group_analysis(d_A2)
print(A1_age_group)
print(A2_age_group)

In [ ]:
A1_cause_person=one_factor_person('肇因研判大類別名稱-主要','肇因研判子類別名稱-主要',d_A1)
A2_cause_person=one_factor_person('肇因研判大類別名稱-主要','肇因研判子類別名稱-主要',d_A2)
cause_person=merge_count(A1_cause_person,A2_cause_person)
cause_person_simple=consolidate_accidents_cause(cause_person)
print(cause_person_simple)

In [ ]:
A1_weather=one_factor('天候名稱',d_A1)
A2_weather=one_factor('天候名稱',d_A2)
A1_light=one_factor('光線名稱',d_A1)
A2_light=one_factor('光線名稱',d_A2)
print(A1_weather)
print(A1_light)
print(A2_weather)
print(A2_light)

In [ ]:
A1_type_person=one_factor_person('事故類型及型態大類別名稱','事故類型及型態子類別名稱',d_A1)
A2_type_person=one_factor_person('事故類型及型態大類別名稱','事故類型及型態子類別名稱',d_A2)
A1_simple_type_person = simplify_accident_types(A1_type_person)
print(A1_simple_type_person)
A2_simple_type_person = simplify_accident_types(A2_type_person)
print(A2_simple_type_person)

In [ ]:
A1_action_person=one_factor_person('當事者行動狀態大類別名稱','當事者行動狀態子類別名稱',d_A1)
A2_action_person=one_factor_person('當事者行動狀態大類別名稱','當事者行動狀態子類別名稱',d_A2)
simple_A1_action_person = consolidate_vehicle_status(A1_action_person)
print(simple_A1_action_person)
simple_A2_action_person = consolidate_vehicle_status(A2_action_person)
print(simple_A2_action_person)

In [ ]:
A1_light_vehicle=two_factors('光線名稱','當事者區分-類別-大類別名稱-車種',d_A1)
A2_light_vehicle=two_factors('光線名稱','當事者區分-類別-大類別名稱-車種',d_A2)
for i in A1_light_vehicle:
    A1_light_vehicle[i]=consolidate_vehicle_types(A1_light_vehicle[i])
for i in A2_light_vehicle:
    A2_light_vehicle[i]=consolidate_vehicle_types(A2_light_vehicle[i])
print(A1_light_vehicle)
print(A2_light_vehicle)

In [ ]:
A1_light_cause=two_factors_person('光線名稱','肇因研判大類別名稱-主要','肇因研判子類別名稱-主要',d_A1)
A2_light_cause=two_factors_person('光線名稱','肇因研判大類別名稱-主要','肇因研判子類別名稱-主要',d_A2)
for i in A1_light_cause:
    A1_light_cause[i]=consolidate_accidents_cause(A1_light_cause[i])
for i in A2_light_cause:
    A2_light_cause[i]=consolidate_accidents_cause(A2_light_cause[i])
print(A1_light_cause)
print(A2_light_cause)

In [ ]:
A1_time_p2=time_analysis(d_A1_p2)
A2_time_p2=time_analysis(d_A2_p2)
print(A1_time_p2,A2_time_p2,sep='\n')
A1_year_p2=year_analysis(d_A1_p2)
A2_year_p2=year_analysis(d_A2_p2)
print(A1_year_p2)
print(A2_year_p2)
A1_death_p2,A1_injury_p2=casulty_analysis(d_A1_p2)
A2_death_p2,A2_injury_p2=casulty_analysis(d_A2_p2)
print(A1_death_p2,A1_injury_p2)
print(A2_death_p2,A2_injury_p2)
A1_death_year_p2,A1_injury_year_p2=casulty_year_analysis(d_A1_p2)
A2_death_year_p2,A2_injury_year_p2=casulty_year_analysis(d_A2_p2)
print(A1_death_year_p2,A1_injury_year_p2,sep='\n')
print(A2_death_year_p2,A2_injury_year_p2,sep='\n')
A1_vehicle_p2=one_factor('當事者區分-類別-大類別名稱-車種',d_A1_p2)
A2_vehicle_p2=one_factor('當事者區分-類別-大類別名稱-車種',d_A2_p2)
A1_vehicle_simple_p2=consolidate_vehicle_types(A1_vehicle_p2)
A2_vehicle_simple_p2=consolidate_vehicle_types(A2_vehicle_p2)
print(A1_vehicle_simple_p2)
print(A2_vehicle_simple_p2)
A1_year_vehicle_p2=two_factors('發生年度','當事者區分-類別-大類別名稱-車種',d_A1_p2)
A2_year_vehicle_p2=two_factors('發生年度','當事者區分-類別-大類別名稱-車種',d_A2_p2)
simple_A1_vehicle_p2 = {i:consolidate_vehicle_types(A1_year_vehicle_p2[i]) for i in A1_year_vehicle_p2.keys()}
print(simple_A1_vehicle_p2)
simple_A2_vehicle_p2 = {i:consolidate_vehicle_types(A2_year_vehicle_p2[i]) for i in A2_year_vehicle_p2.keys()}
print(simple_A2_vehicle_p2)
A1_vehicle_death_p2=factor_death('當事者區分-類別-大類別名稱-車種',d_A1_p2)
A2_vehicle_death_p2=factor_death('當事者區分-類別-大類別名稱-車種',d_A2_p2)
A1_vehicle_injury_p2=factor_injury('當事者區分-類別-大類別名稱-車種',d_A1_p2)
A2_vehicle_injury_p2=factor_injury('當事者區分-類別-大類別名稱-車種',d_A2_p2)
simple_A1_vehicle_death_p2 = consolidate_vehicle_types(A1_vehicle_death_p2)
print(simple_A1_vehicle_death_p2)
simple_A2_vehicle_injury_p2 = consolidate_vehicle_types(A2_vehicle_injury_p2)
print(simple_A2_vehicle_injury_p2)
A1_age_p2=one_factor('當事者事故發生時年齡',d_A1_p2)
A2_age_p2=one_factor('當事者事故發生時年齡',d_A2_p2)
A1_age_group_p2=age_group_analysis(d_A1_p2)
A2_age_group_p2=age_group_analysis(d_A2_p2)
print(A1_age_group_p2)
print(A2_age_group_p2)
A1_cause_person_p2=one_factor_person('肇因研判大類別名稱-主要','肇因研判子類別名稱-主要',d_A1_p2)
A2_cause_person_p2=one_factor_person('肇因研判大類別名稱-主要','肇因研判子類別名稱-主要',d_A2_p2)
cause_person_p2=merge_count(A1_cause_person_p2,A2_cause_person_p2)
cause_person_simple_p2=consolidate_accidents_cause(cause_person_p2)
print(cause_person_simple_p2)
A1_weather_p2=one_factor('天候名稱',d_A1_p2)
A2_weather_p2=one_factor('天候名稱',d_A2_p2)
A1_light_p2=one_factor('光線名稱',d_A1_p2)
A2_light_p2=one_factor('光線名稱',d_A2_p2)
print(A1_weather_p2)
print(A1_light_p2)
print(A2_weather_p2)
print(A2_light_p2)
A1_type_person_p2=one_factor_person('事故類型及型態大類別名稱','事故類型及型態子類別名稱',d_A1_p2)
A2_type_person_p2=one_factor_person('事故類型及型態大類別名稱','事故類型及型態子類別名稱',d_A2_p2)
A1_simple_type_person_p2 = simplify_accident_types(A1_type_person_p2)
print(A1_simple_type_person_p2)
A2_simple_type_person_p2 = simplify_accident_types(A2_type_person_p2)
print(A2_simple_type_person_p2)
A1_action_person_p2=one_factor_person('當事者行動狀態大類別名稱','當事者行動狀態子類別名稱',d_A1_p2)
A2_action_person_p2=one_factor_person('當事者行動狀態大類別名稱','當事者行動狀態子類別名稱',d_A2_p2)
simple_A1_action_person_p2 = consolidate_vehicle_status(A1_action_person_p2)
print(simple_A1_action_person_p2)
simple_A2_action_person_p2 = consolidate_vehicle_status(A2_action_person_p2)
print(simple_A2_action_person_p2)
A1_light_vehicle_p2=two_factors('光線名稱','當事者區分-類別-大類別名稱-車種',d_A1_p2)
A2_light_vehicle_p2=two_factors('光線名稱','當事者區分-類別-大類別名稱-車種',d_A2_p2)
print(A1_light_vehicle_p2)
print(A2_light_vehicle_p2)

In [ ]:
print('車種')
print(merge_count(A1_vehicle_simple,A2_vehicle_simple))
print(merge_count(A1_vehicle_simple_p2,A2_vehicle_simple_p2))
veh_merge=merge_count(A1_vehicle_simple,A2_vehicle_simple)
veh_merge_p2=merge_count(A1_vehicle_simple_p2,A2_vehicle_simple_p2)
del veh_merge['未知或未分類']
del veh_merge_p2['未知或未分類']
print(factor_diff_significance(veh_merge,veh_merge_p2))
print()
print('時間')
print(merge_count(A1_time,A2_time))
print(merge_count(A1_time_p2,A2_time_p2))
t1=[116492, 174147, 158690, 121412, 117336, 126762, 217530, 101319, 111517, 131411, 181721, 140025, 100658, 227823]
t2=[19, 16, 13, 14, 23, 28, 47, 30, 32, 24, 18, 12, 9, 14]
t1_pct=[i*100/sum(t1) for i in t1]
t2_pct=[i*100/sum(t2) for i in t2]
chi_square_stat, p_value = chisquare(t2_pct, f_exp=t1_pct)
if p_value<0.05:
    print(f'有顯著差異，p-value為{p_value:.4f}')
else:   
    print(f'無顯著差異，p-value為{p_value:.4f}')
print()
print('年齡')
print(merge_count(A1_age_group,A2_age_group))
print(merge_count(A1_age_group_p2,A2_age_group_p2))
print(factor_diff_significance(merge_count(A1_age_group,A2_age_group),merge_count(A1_age_group_p2,A2_age_group_p2),print_oe=True))
print()
print('事故原因')
print(cause_person_simple)
print(cause_person_simple_p2)
cause_person_simple_eliminate=cause_person_simple.copy()
cause_person_simple_p2_eliminate=cause_person_simple_p2.copy()
del cause_person_simple_eliminate['其他']
del cause_person_simple_p2_eliminate['其他']
del cause_person_simple_eliminate['不明']
del cause_person_simple_p2_eliminate['不明']
print(factor_diff_significance(cause_person_simple_eliminate,cause_person_simple_p2_eliminate))
print()
print('天氣')
print(merge_count(A1_weather,A2_weather))
print(merge_count(A1_weather_p2,A2_weather_p2))
print(factor_diff_significance(merge_count(A1_weather,A2_weather),merge_count(A1_weather_p2,A2_weather_p2)))
print()
print('光線')
print(merge_count(A1_light,A2_light))
print(merge_count(A1_light_p2,A2_light_p2))
print(factor_diff_significance(merge_count(A1_light,A2_light),merge_count(A1_light_p2,A2_light_p2)))
print()
print('事故類型')
print(merge_count(A1_simple_type_person,A2_simple_type_person))
print(merge_count(A1_simple_type_person_p2,A2_simple_type_person_p2))
type_person_merge=merge_count(A1_simple_type_person,A2_simple_type_person)
type_person_merge_p2=merge_count(A1_simple_type_person_p2,A2_simple_type_person_p2)
del type_person_merge['其他']
del type_person_merge_p2['其他']
del type_person_merge['車與車事故_其他']
del type_person_merge_p2['車與車事故_其他']
print(factor_diff_significance(merge_count(A1_simple_type_person,A2_simple_type_person),merge_count(A1_simple_type_person_p2,A2_simple_type_person_p2)))
print()
print('行動狀態')
print(merge_count(simple_A1_action_person,simple_A2_action_person))
print(merge_count(simple_A1_action_person_p2,simple_A2_action_person_p2))
print(factor_diff_significance(merge_count(simple_A1_action_person,simple_A2_action_person),merge_count(simple_A1_action_person_p2,simple_A2_action_person_p2)))